## Index Tutorial for NYC Neighborhood Tabulation Areas


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danmillr/places-platforms/blob/main/02_Index/02_Index.ipynb)


---

## 1. Overview & Learning Objectives
This tutorial guides you through constructing a spatial index using real-world data from New York City. We will build a **Heat Vulnerability Index (HVI)** that combines environmental and demographic indicators.

You will:
- Learn how spatial indices are used for policy and planning
- Join multiple spatial and tabular datasets
- Normalize variables to prepare them for aggregation
- Build a weighted composite index with dynamic inputs
- Create static and interactive maps
- Critically evaluate the assumptions behind your index

By the end, you should be able to construct your own index and reflect on how methodological decisions shape outcomes.

---

## 2. Setup
Install and import the required libraries. These include pandas and geopandas for data handling, matplotlib and folium for visualization, and ipywidgets for interactivity.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium import Choropleth
import ipywidgets as widgets
from IPython.display import display, Markdown

---

## 3. Data Overview
We are using four datasets:
1. **NTA GeoJSON** – geographic boundaries for Neighborhood Tabulation Areas
2. **Heat Vulnerability Index (HVI)** – a precomputed score reflecting environmental risk factors
3. **Household Crowding** – % of households with more than one person per room
4. **Canopy/Shade** – a proxy for protection from heat based on tree cover

These datasets are at the same spatial level (NTA) or have identifiers (`GEOCODE`) that allow matching.

In [ ]:
nta_gdf = gpd.read_file("https://github.com/danmillr/places-platforms/blob/main/02_Index/Data/2020%20Neighborhood%20Tabulation%20Areas%20(NTAs)_20250626.geojson")
hvi_df = pd.read_csv("path_to/hvi-nta-2020.csv")
crowding_df = pd.read_csv("https://github.com/danmillr/places-platforms/blob/main/02_Index/Data/nta_crowding.csv")
canopy_df = pd.read_csv("https://github.com/danmillr/places-platforms/blob/main/02_Index/Data/canopystreettree_supp_nta.csv")

---

## 4. Data Linking & Validation
This step merges the datasets into one table. The keys used are `NTACode` and `GEOCODE`. Always inspect your joins to ensure you aren't introducing nulls or losing rows.

In [ ]:
crowding_df = crowding_df.rename(columns={"GeoID": "GEOCODE"})
canopy_df["NTACode"] = canopy_df["ntacode"].str.upper()

merged_df = hvi_df.merge(crowding_df, on="GEOCODE", how="left")
merged_df = merged_df.merge(canopy_df, on="NTACode", how="left")
merged_gdf = nta_gdf.merge(merged_df, on="NTACode", how="left")

---

## 5. Exploratory Data Analysis
We explore variable distributions to identify skew or outliers. This step also helps students reason about transformations needed before constructing the index.

In [ ]:
merged_gdf['Percent'].dropna().astype(float).hist(bins=20)
plt.title("Crowding Rate Distribution")
plt.xlabel("Percent Crowded")
plt.ylabel("Number of NTAs")
plt.show()

Plotting a map of the raw variable:

In [ ]:
merged_gdf.plot(column='Percent', cmap='OrRd', legend=True, figsize=(10,6))
plt.title("Crowding Rate by NTA")
plt.axis('off')
plt.show()

---

## 6. Normalization
To combine variables, they must be on the same scale. We use **min–max normalization**:

\[ x_{norm} = 
rac{x - x_{min}}{x_{max} - x_{min}} \]

This is useful for visualization and additive indices but sensitive to outliers.

In [ ]:
variables = ['Percent', 'SURFACE_TEMP', 'canopy_pct']
for var in variables:
    merged_gdf[f"{var}_norm"] = (
        merged_gdf[var] - merged_gdf[var].min()
    ) / (merged_gdf[var].max() - merged_gdf[var].min())

---

## 7. Build Your Index (Interactive)
You can set your own weights to explore different assumptions about which factors contribute most to heat vulnerability. Note that canopy is inverted to represent less vulnerability with more coverage.

In [ ]:
w_crowd = widgets.FloatSlider(0.33, 0, 1, 0.01, description='Crowding')
w_temp = widgets.FloatSlider(0.33, 0, 1, 0.01, description='Temperature')
w_canopy = widgets.FloatSlider(0.33, 0, 1, 0.01, description='Canopy')

def update_index(crowd, temp, canopy):
    total = crowd + temp + canopy
    merged_gdf['custom_index'] = (
        crowd * merged_gdf['Percent_norm'] +
        temp * merged_gdf['SURFACE_TEMP_norm'] +
        canopy * (1 - merged_gdf['canopy_pct_norm'])
    ) / total

    ax = merged_gdf.plot(
        column='custom_index', cmap='plasma', legend=True, figsize=(10,6)
    )
    plt.title("Custom Heat Vulnerability Index")
    plt.axis('off')
    plt.show()

widgets.interact(update_index, crowd=w_crowd, temp=w_temp, canopy=w_canopy);

---

## 8. Advanced Options
Explore advanced methods to improve, validate, or challenge your index construction. These techniques go beyond static weightings and help illuminate structure in your data.

**Alternative normalization:** Use z-score normalization to standardize variables based on their distance from the mean.

In [ ]:
from scipy.stats import zscore
merged_gdf['Percent_z'] = zscore(merged_gdf['Percent'].dropna())

This is particularly useful if your variables are skewed or have outliers, as it centers data at zero with a standard deviation of one.

**Add more variables:** You can enrich the index by including:
- % of residents over 65 (age vulnerability)
- % without air conditioning (exposure risk)
- % non-English speakers (language isolation)
- Access to public cooling centers or shaded green space

Make sure to normalize any new variables before combining them.

**PCA (Principal Component Analysis):** PCA reduces multiple related variables into components that capture the most variance. It can be used to simplify and weight input dimensions empirically.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

features = ['Percent_norm', 'SURFACE_TEMP_norm', 'canopy_pct_norm']
X = StandardScaler().fit_transform(merged_gdf[features].dropna())
pca = PCA(n_components=1)
merged_gdf['pca_index'] = pca.fit_transform(X)

**Clustering:** Use algorithms like KMeans to segment NTAs into distinct groups of vulnerability, rather than using a single continuous index.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
merged_gdf['cluster'] = kmeans.labels_

These tools allow you to compare your subjective weightings with data-driven structures. They can help expose hidden patterns or support alternative interpretations of vulnerability.

---

## 9. Export Your Results
Export your final output as a GeoJSON to use in GIS or for sharing.

In [ ]:
merged_gdf[['NTACode', 'custom_index', 'geometry']].to_file("custom_hvi.geojson", driver="GeoJSON")

---

## 10. Critical Reflection
Questions for consideration:
- What kinds of vulnerability are not captured here?
- How do your weights reflect your assumptions?
- Are there ethical concerns with publicly mapping vulnerability?
- What might a participatory or community-informed index look like?

Indexes are powerful—but they are never neutral. Be reflective, transparent, and critical.